In [1]:
import geopandas as gpd
import folium
from folium import plugins
import pandas as pd

In [2]:
input_file = 'observations/observations.shp'
output_file = 'mapa_krukow_finlandia_2021.html'

In [3]:
col_date = 'date'
col_count = 'amount'
col_observer = 'observer'

In [4]:
try :
    gdf = gpd.read_file(input_file)
except Exception as e :
    print(f"Nie znaleziono pliku: {input_file}!")
    exit()

In [5]:
gdf.set_crs(epsg=4326, inplace=True)
gdf[col_count] = gdf[col_count].astype(int)

In [6]:
m = folium.Map(
    location=[64.9146659, 26.0672554],
    zoom_start=5,
    tiles='CartoDB positron',
    control_scale=True
)

In [7]:
layer_dates = folium.FeatureGroup(name='Daty obserwacji', show=False)

for idx, row in gdf.iterrows() :
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        tooltip=f"Data obserwacji: {row[col_date]}"
    ).add_to(layer_dates)

layer_dates.add_to(m)

heat_data = [[row.geometry.y, row.geometry.x, row[col_count]] for idx, row in gdf.iterrows()]

In [8]:
layer_heatmap = folium.FeatureGroup(name="Mapa ciepła", show=False)
plugins.HeatMap(
    heat_data,
    name="Mapa ciepła",
    radius=15,
    blur=10,
    max_zoom=10
).add_to(layer_heatmap)

layer_heatmap.add_to(m)

layer_clusters = folium.FeatureGroup(name="Szczegóły", show=False)
marker_cluster = plugins.MarkerCluster().add_to(layer_clusters)


In [9]:
unique_observers = gdf[col_observer].unique()
colors = ['red', 'green', 'blue', 'orange', 'purple', 'cadetblue', 'darkred']
observer_color_map = {obs : colors[i % len(colors)] for i, obs in enumerate(unique_observers)}

for idx, row in gdf.iterrows() :
    count = row[col_count]
    observer = row[col_observer]

    if count == 1:
        icon_name = 'cube'
        icon_prefix = 'fa'
    else:
        icon_name = 'cubes'
        icon_prefix = 'fa'

    popup_html = f"""
        <div style="font-family: Arial; min-width: 150px;">
            <b>Obserwator:</b> {observer}<br>
            <b>Liczba ptaków:</b> {count}<br>
            <hr style="margin: 5px 0;">
            <small>Data: {row[col_date]}</small>
        </div>
    """

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        tooltip=f"Kliknij, żeby zobaczyć szczegóły",
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(
            color=observer_color_map.get(observer, 'gray'),
            icon=icon_name,
            prefix=icon_prefix
        )
    ).add_to(marker_cluster)

layer_clusters.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save(output_file)